# Prompt Management with Vertex AI

This notebook demonstrates how to create, manage, and use prompt templates with Vertex AI's prompt management features. Prompt management allows you to version, share, and reuse prompts across your organization.

## 1. Initialize Vertex AI

First, we set up the project ID and location, then initialize the Vertex AI client to access Vertex AI services.

In [ ]:
PROJECT_ID = "sascha-playground-doit"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

## 2. Import Required Libraries

We import the necessary modules from the Vertex AI SDK, including the prompts modules that allow us to create and manage prompt templates.

In [ ]:
import vertexai
from vertexai.preview import prompts
from vertexai.preview.prompts import Prompt

## 3. Create a Local Prompt Template

Here we create a local prompt template for artist name matching. This template includes:
- A prompt name for identification
- The main prompt text with a variable placeholder {artist}
- Example variables to substitute into the prompt
- The Gemini model to use
- A detailed system instruction that guides the model's behavior

Note that this prompt remains local until we explicitly save it to Vertex AI.

In [ ]:
# this prompt is local until saved
prompt = Prompt(
    prompt_name="artist-matcher",
    prompt_data="correct the following artist {artist}",
    variables=[
        {"artist": "acdc"},
        {"artist": "Pink Floy"},
    ],
    model_name="gemini-2.0-flash-001",
    system_instruction="""
    you are a artist matching services based on a list of artists provided you suggest the correct naming. users might use slightly different names you need to ensure they are exactly as in the artist list

    artists:
    The Rolling Stones
    The Beatles
    Led Zeppelin
    Pink Floyd
    The Who
    The Doors
    Queen
    Aerosmith
    The Eagles
    Fleetwood Mac
    David Bowie
    Jimi Hendrix
    Bob Dylan
    AC/DC
    Guns N' Roses
    The Clash
    Nirvana
    U2
    Bruce Springsteen
    The Kinks
    The Beach Boys
    """,
)

## 4. Save the Prompt to Vertex AI

We now save the local prompt to Vertex AI's prompt management system using the create_version() function. This creates a persistent, versioned prompt resource in Vertex AI that can be shared and reused. The function returns a new Prompt object associated with the online resource.

In [ ]:
# Save a version
prompt = prompts.create_version(prompt=prompt)
prompt

## 5. Retrieving a Saved Prompt

To access a prompt that was previously saved, we can use the get() method with the prompt ID. This allows you to retrieve prompt templates created by you or shared within your organization.

In [ ]:
prompt = prompts.get("8464170802747539456")  #prompt id from UI
prompt

## 6. Listing All Available Prompts

The list() method helps you discover all prompt templates available in your project. This is useful for finding existing prompts that you or your team have created.

In [ ]:
prompts.list()

## 7. Using a Prompt with the Gemini Model

Now we use our saved prompt to generate content with Gemini. We can either pass a direct input or use the prompt's variables. The system instructions we defined earlier guide the model to correct artist names according to our reference list.

In [ ]:

response = prompt.generate_content(
    "acdc"
)

print(response)
print(response.candidates[0].content.parts[0].text)

## 8. Updating an Existing Prompt

To update a prompt, you modify its properties locally and then create a new version with create_version(). This preserves the version history while adding the updated version as the latest one. Here we change the system instruction and save a new version.

In [ ]:


# Generate content using the assembled prompt (a prompt without variables)
prompt.generate_content(
  contents=prompt.assemble_contents()
)

# Update prompt (changes are local until create_version is called)
prompt.system_instruction = "new system instruction"

# Save Prompt to online resource. Since the prompt is associated with a prompt resource, it creates a new version under the same prompt_id. Returns a new Prompt object associated with the online resource
prompts.create_version(prompt=prompt)

## 9. Listing Prompt Versions

Vertex AI maintains version history for each prompt. You can list all versions of a prompt using the list_versions() method, which returns metadata about each version including its ID and creation timestamp.

In [ ]:
prompt_versions = prompts.list_versions(prompt_id="4657221742735917056")
prompt_versions

## 10. Retrieving a Specific Prompt Version

You can retrieve a specific version of a prompt by providing both the prompt ID and version ID. This allows you to use older versions if needed or compare different versions of the same prompt.

In [ ]:
# Get a specific prompt version from the versions metadata list
prompt1 = prompts.get(
    prompt_id=prompt_versions[2].prompt_id,
    version_id=prompt_versions[2].version_id
)
prompt1